##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# モデルの保存と復元

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_restore_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

モデルは訓練中にも、訓練が終わったあとも保存できます。このことは、長い訓練時間を掛けなくても、やめたところから再開できるということを意味します。モデルが保存可能であることは、あなたが作ったモデルを他の人と共有できるということでもあります。研究結果であるモデルや手法を公開する際、機械学習の実務家はほとんど次のものを共有します。

* モデルを構築するプログラム
* 学習済みモデルの重みあるいはパラメータ

このデータを共有することで、他の人がモデルだどの様に動作するかを理解したり、新しいデータに試してみたりすることが容易になります。

注意：信頼できないプログラムには気をつけましょう。TensorFlowのモデルもプログラムです。詳しくは、[Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)を参照してください。

### オプション

TensorFlowのモデルを保存する方法は、使っているAPIによって異なります。このガイドはTensorFlowのモデルを構築し訓練するためのハイレベルなAPIである[tf.keras](https://www.tensorflow.org/guide/keras)を使っています。この他のアプローチについては、TensorFlowの [Save and Restore](https://www.tensorflow.org/guide/saved_model) ガイド、あるいは、[Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving)を参照してください。

## 設定

### インストールとインポート

TensorFlowと依存関係のライブラリをインストールし、インポートします。

In [ ]:
!pip install h5py pyyaml 

### サンプルデータセットの取得

ここでは、モデルを訓練し重みの保存をデモするために、 [MNIST dataset](http://yann.lecun.com/exdb/mnist/) を使います。デモの実行を速くするため、最初の1,000件のサンプルだけを使います。

In [ ]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### モデルの定義

重みの保存と読み込みのデモを行うための簡単なモデルを定義しましょう。

In [ ]:
# 短いシーケンシャルモデルを返す関数
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer='adam', 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# 基本的なモデルのインスタンスを作る
model = create_model()
model.summary()

## 訓練中にチェックポイントを保存する

主な用途は訓練の**途中**あるいは**終了後**にチェックポイントを自動的に保存することです。こうすることにより、再び訓練を行うことなくモデルを使用することができ、また、訓練が中断された場合に、中止したところから再開できます。

`tf.keras.callbacks.ModelCheckpoint`がこれを行うためのコールバックです。このコールバックにはチェックポイントを構成するためのいくつかの引数があります。

### チェックポイントコールバックの使い方

モデルの訓練時に、`ModelCheckpoint`を渡します。

In [24]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 訓練にコールバックを渡す

# オプティマイザの状態保存についての警告が表示されるかもしれません。
# これらの警告は（このノートブックで発生する同様な警告を含めて）
# 古い用法を非推奨にするためのもので、無視して構いません。

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 928/1000 [==========================>...] - ETA: 0s - loss: 1.2115 - acc: 0.6616
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1684 - acc: 0.6750 - val_loss: 0.6991 - val_acc: 0.8030
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4275 - acc: 0.8729
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 297us/sample - loss: 0.4226 - acc: 0.8750 - val_loss: 0.5036 - val_acc: 0.8550
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2743 - acc: 0.9256
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 297us/sample - loss: 0.2830 - acc: 0.9240 - val_loss: 0.4931 - val_acc: 0.8420
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2130 - acc: 0.9510
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000

この結果、エポックごとに更新される一連のTensorFlowチェックポイントファイルが作成されます。

In [25]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


訓練していない新しいモデルを作ります。重みだけからモデルを復元する場合には、元のモデルと同じアーキテクチャのモデルが必要です。モデルのアーキテクチャが同じであるため、モデルの異なる**インスタンス**であっても重みを共有することができるのです。

訓練していない全く新しいモデルを作り、テストデータセットで評価します。訓練をしていないモデルは偶然のレベル（正解率10%以下）の性能しか無いはずです。

In [26]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 187us/sample - loss: 2.3399 - acc: 0.1430
Untrained model, accuracy: 14.30%


次に、チェックポイントから重みをロードし、再び評価します。

In [27]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 72us/sample - loss: 0.4049 - acc: 0.8680
Restored model, accuracy: 86.80%


### チェックポイントコールバックのオプション

このコールバックには、チェックポイントに一意な名前をつけたり、チェックポイントの頻度を調整するためのオプションがあります。

新しいモデルを訓練し、5エポックごとに一意な名前のチェックポイントを保存します。

In [28]:
# include the epoch in the file name. (uses `str.format`)
# ファイル名に(str.format`を使って)エポック数を埋め込みます
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 重みを5エポックごとに保存します
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


次に、出来上がったチェックポイントを確認し、最後のものを選択します。

In [29]:
! ls {checkpoint_dir}

checkpoint                       cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001 cp-0030.ckpt.index
cp-0005.ckpt.index               cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001 cp-0035.ckpt.index
cp-0010.ckpt.index               cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001 cp-0040.ckpt.index
cp-0015.ckpt.index               cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001 cp-0045.ckpt.index
cp-0020.ckpt.index               cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001 cp-0050.ckpt.index
cp-0025.ckpt.index


In [30]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

注意：デフォルトのtensorflowフォーマットは、直近の5つのチェックポイントのみを保存します。

テストのため、モデルをリセットし最後のチェックポイントをロードします。

In [31]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 245us/sample - loss: 0.4864 - acc: 0.8710
Restored model, accuracy: 87.10%


## これらのファイルは何？

上記のコードでは、重みだけをバイナリで[checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)形式の一連のファイルに保存します。チェックポイントには、次のものが含まれます。

* １つ以上のモデルの重みの断片
* どの重みがどの断片に保存されているかを示すインデックスファイル

1台のマシンだけでモデルの訓練を行っている場合には、`.data-00000-of-00001`のようなサフィックスのついたファイルが1つだけ作成されます。

## 手動で重みを保存する

上記では重みをモデルにロードする方法を見ました。

手動で重みを保存するのも同じ様に簡単です。`Model.save_weights` メソッドを使います。

In [32]:
# 重みの保存
model.save_weights('./checkpoints/my_checkpoint')

# 重みの復元
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 213us/sample - loss: 0.4864 - acc: 0.8710
Restored model, accuracy: 87.10%


## モデル全体の保存

重みの値、モデルの設定、（構成によっては）オプティマイザの設定までも含んだモデル全体をファイルに保存することができます。これにより、モデルのチェックポイントを保存し、オリジナルのコードにアクセスする必要なく、中断したところから訓練を再開することができます。

完全に機能するモデルを保存できるのは便利です。保存したモデルをTensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html))でロードし、ブラウザで訓練したり、実行したりすることができるほか、TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))
を使ってモバイルデバイスで実行できるように変換することも可能です。

### HDF5ファイルとして

Kerasでは、[HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 標準を使った基本的なファイルフォーマットが利用できます。ここでの利用目的では、保存されたモデルは単独のバイナリラージオブジェクト（blob）として扱うことができます。

In [33]:
model = create_model()

# HDF5ファイルにオプティマイザの状態を保存するにはkeras.optimaizerを使用する必要があります
model.compile(optimizer='adam', 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

# モデル全体をHDF5ファイルに保存
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 343us/sample - loss: 1.2038 - acc: 0.6630
Epoch 2/5
1000/1000 [==============================] - 0s 157us/sample - loss: 0.4228 - acc: 0.8930
Epoch 3/5
1000/1000 [==============================] - 0s 157us/sample - loss: 0.2828 - acc: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 152us/sample - loss: 0.2314 - acc: 0.9300
Epoch 5/5
1000/1000 [==============================] - 0s 155us/sample - loss: 0.1627 - acc: 0.9650


保存したファイルを使ってモデルを再作成します。

In [34]:
# 重みとオプティマイザを含む全く同じモデルを再作成
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


正解率を検査します。

In [35]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 218us/sample - loss: 0.4583 - acc: 0.8530
Restored model, accuracy: 85.30%


この方法では、次のすべてが保存されます。

* 重みの値
* モデルの設定（アーキテクチャ）
* オプティマイザの設定

Kerasは保存する際にアーキテクチャを調べます。いまのところ、TensorFlowのオプティマイザ（`tf.train`に含まれるもの）を保存することはできません。TensorFlowのオプティマイザを使用している場合には、モデルをロードしたあと再コンパイルする必要があり、オプティマイザの状態は失われます。

### `saved_model`として

注意：この手法による`tf.keras`モデルの保存は実験的なもので、将来のバージョンで変更される可能性があります。

新しいモデルを作ります。

In [36]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 388us/sample - loss: 1.1508 - acc: 0.6660
Epoch 2/5
1000/1000 [==============================] - 0s 161us/sample - loss: 0.4185 - acc: 0.8880
Epoch 3/5
1000/1000 [==============================] - 0s 176us/sample - loss: 0.2884 - acc: 0.9300
Epoch 4/5
1000/1000 [==============================] - 0s 170us/sample - loss: 0.2032 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 163us/sample - loss: 0.1529 - acc: 0.9650


`saved_model`を作成します。

In [37]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")

AssertionError: Export directory already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory: ./saved_models

セーブドモデル（saved models） はタイムスタンプ付きのディレクトリに保存されます。

In [38]:
!ls saved_models/

assets         saved_model.pb variables


セーブドモデルから新しいKerasモデルをリロードします。

In [39]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model

TypeError: Expected binary or unicode string, got None

復元されたモデルを実行します。

In [40]:
# The optimizer was not restored, re-attach a new one.
# オプティマイザは復元されません。新しいオプティマイザを設定し直します。
new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 234us/sample - loss: 0.4583 - acc: 0.8530
Restored model, accuracy: 85.30%


## この先は？

`tf.keras`を使った保存とロードのクイックガイドでした。

* [tf.keras guide](https://www.tensorflow.org/guide/keras) には`tf.keras`での保存とロードについて、もう少し記載されています

* Eager Executionでの保存については[Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving) を参照ください

* [Save and Restore](https://www.tensorflow.org/guide/saved_model)ガイドには、TensorFlowでの保存についてローレベルの詳細が記載されています